In [24]:
%load_ext autoreload
%autoreload 2

# import cv2
from tqdm.notebook import tqdm
import sys
from IPython.display import display, Markdown
import plotly.express as px
from plotly import subplots as sp
from tensorboardX import SummaryWriter

import torch
from torch.utils.data import DataLoader

sys.path.append('../../')
sys.path.append('../')
from scikitools.deep_learning.dataload import LocalImageLoader
from src.celeb_a import CELEBACNNDiscriminator, CELEBACNNGenerator
from src.generator import (
    generate_random_image,
    generate_random_seed
)

device = torch.device("cuda")
torch.cuda.is_available()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [16]:
celeb_datasets = LocalImageLoader(
    '../../../../../datasets/celeba/img_align_celeba/img_align_celeba',
    crop_size=(128, 128)
)

In [17]:
celeb_datasets.plot_image(0, True).show()
celeb_datasets.plot_image(0, False).show()

torch.Size([128, 128, 3])


torch.Size([178, 218, 3])


In [4]:
d = CELEBACNNDiscriminator()
d.to(device)

celeb_dataloader = DataLoader(celeb_datasets, num_workers=5, prefetch_factor=10)

for image_data_tensor, _ in tqdm(celeb_datasets):
    d.train(image_data_tensor.view(1, 3, 128, 128).cuda().float()/255., torch.cuda.FloatTensor([1.0]))
    d.train(
        generate_random_image((1, 3, 128, 128)).cuda(),
        torch.FloatTensor([0.0]).cuda()
    )

  0%|          | 0/202599 [00:00<?, ?it/s]

counter = 10000
counter = 20000
counter = 30000
counter = 40000
counter = 50000
counter = 60000
counter = 70000
counter = 80000
counter = 90000
counter = 100000
counter = 110000
counter = 120000
counter = 130000
counter = 140000
counter = 150000
counter = 160000
counter = 170000
counter = 180000
counter = 190000
counter = 200000
counter = 210000
counter = 220000
counter = 230000
counter = 240000
counter = 250000
counter = 260000
counter = 270000
counter = 280000
counter = 290000
counter = 300000
counter = 310000
counter = 320000
counter = 330000
counter = 340000
counter = 350000
counter = 360000
counter = 370000
counter = 380000
counter = 390000
counter = 400000


In [5]:
d.plot_progress()

In [34]:
g = CELEBACNNGenerator()
g.to(device)
output = g.forward(generate_random_seed(100).cuda())
img = output.detach().permute(0,2,3,1).view(128,128,3).cpu().numpy()
px.imshow(img)

(128, 128, 3)


In [40]:
d = CELEBACNNDiscriminator()
g = CELEBACNNGenerator()

d.to(device)
g.to(device)

epochs = 4

writer = SummaryWriter("celebA-CNN-GAN-log")

with tqdm(total=epochs*len(celeb_datasets)) as pbar:
    for epoch in range(epochs):
        print(f"epcho = {epoch+1}")
        for i, image_data_tensor_with_label in enumerate(celeb_datasets):
            image_data_tensor, _ = image_data_tensor_with_label
            d.train(
                image_data_tensor.view(1, 3, 128, 128).cuda().float()/255.,
                torch.cuda.FloatTensor([1.0])
                )
            # d.train(
            #     generate_random_image((1, 3, 128, 128)).cuda(),
            #     torch.FloatTensor([0.0]).cuda()
            # )
            d.train(
                g.forward(generate_random_seed(100).cuda()).detach(),
                torch.FloatTensor([0.0]).cuda()
            )
            g.train(
                d,
                generate_random_seed(100).cuda(),
                torch.FloatTensor([1.0]).cuda()
            )
            writer.add_scalar('Discriminator/Loss', d.temp_loss, g.counter)
            writer.add_scalar('Generator/loss', g.temp_loss, g.counter)
            pbar.update(1)
writer.close()

  0%|          | 0/810396 [00:00<?, ?it/s]

epcho = 1
counter = 10000
counter = 20000
counter = 30000
counter = 40000
counter = 50000
counter = 60000
counter = 70000
counter = 80000
counter = 90000
counter = 100000
counter = 110000
counter = 120000
counter = 130000
counter = 140000
counter = 150000
counter = 160000
counter = 170000
counter = 180000
counter = 190000
counter = 200000
counter = 210000
counter = 220000
counter = 230000
counter = 240000
counter = 250000
counter = 260000
counter = 270000
counter = 280000
counter = 290000
counter = 300000
counter = 310000
counter = 320000
counter = 330000
counter = 340000
counter = 350000
counter = 360000
counter = 370000
counter = 380000
counter = 390000
counter = 400000
epcho = 2
counter = 410000
counter = 420000
counter = 430000
counter = 440000
counter = 450000
counter = 460000
counter = 470000
counter = 480000
counter = 490000
counter = 500000
counter = 510000
counter = 520000
counter = 530000
counter = 540000
counter = 550000
counter = 560000
counter = 570000
counter = 580000
cou

In [41]:
fig_with_sub = sp.make_subplots(rows=3, cols=3)
for i in range(1, 4):
    for j in range(1, 4):
        img = (g.forward(generate_random_seed(100).cuda())
                .detach()
                .cpu()
                .numpy().reshape((3, 128, 128))
                .transpose(1, 2, 0))
        fig = px.imshow(
                img
            )
        fig_with_sub.append_trace(
            fig.data[0],
            row=i, col=j
        )
fig_with_sub.show()